In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D

import numpy as np
from glob import glob

batch = 32
img_height = 180
img_width = 180

train_path = r"D:\Project new  dataset\Test"
test_path = r"D:\Project new  dataset\Test"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_path,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch
)

class_names = train_ds.class_names
print(class_names)

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers.experimental import preprocessing

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Normalize data by using a rescaling layer
normalization_layer = preprocessing.Rescaling(1.0 / 255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

# Freeze the layers of the pre-trained model
base_model.trainable = False

# Create a custom classification head
Classifier = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(units=128, activation='relu'),
    layers.Dense(units=7, activation='softmax')
])

Classifier.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

# Fit the classifier
epochs = 60
r = Classifier.fit(
    train_ds,
    validation_data=test_ds,
    epochs=epochs,
)


Found 42 files belonging to 7 classes.
Found 42 files belonging to 7 classes.
['1Hundrednote', '2Hundrednote', '2Thousandnote', '5Hundrednote', 'Fiftynote', 'Tennote', 'Twentynote']
9406464/9406464 [==============================] - 3s 0us/step
Epoch 1/60


C:\Users\CHERUKURI GUNNESH\anaconda3\lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2/2 [==============================] - 11s 4s/step - loss: 2.0745 - accuracy: 0.1190 - val_loss: 1.5953 - val_accuracy: 0.3810
Epoch 2/60
2/2 [==============================] - 2s 2s/step - loss: 1.7522 - accuracy: 0.2857 - val_loss: 1.3375 - val_accuracy: 0.4524
Epoch 3/60
2/2 [==============================] - 2s 1s/step - loss: 1.3506 - accuracy: 0.4524 - val_loss: 0.9964 - val_accuracy: 0.7143
Epoch 4/60
2/2 [==============================] - 2s 1s/step - loss: 0.9918 - accuracy: 0.7381 - val_loss: 0.8161 - val_accuracy: 0.8571
Epoch 5/60
2/2 [==============================] - 2s 2s/step - loss: 0.7729 - accuracy: 0.9286 - val_loss: 0.6657 - val_accuracy: 0.9524
Epoch 6/60
2/2 [==============================] - 2s 1s/step - loss: 0.6516 - accuracy: 0.9524 - val_loss: 0.5734 - val_accuracy: 0.9286
Epoch 7/60
2/2 [==============================] - 2s 2s/step - loss: 0.5732 - accuracy: 0.8810 - val_loss: 0.4946 - val_accuracy: 0.9048
Epoch 8/60
2/2 [==============================] - 2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers.experimental import preprocessing

AUTOTUNE = tf.data.AUTOTUNE

# Define your data loading and preprocessing here

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

# Freeze the layers of the pre-trained model
base_model.trainable = False

# Create a custom classification head
Classifier = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(units=64, activation='relu'),  # Reduced units in the dense layer
    layers.Dense(units=7, activation='softmax')
])

Classifier.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

# Reduce the number of epochs
epochs = 50

# Use aggressive data augmentation
data_augmentation = tf.keras.Sequential([
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomContrast(0.2),
])

# Fit the classifier
r = Classifier.fit(
    train_ds.map(lambda x, y: (data_augmentation(x), y)),  # Apply data augmentation
    validation_data=test_ds,
    epochs=epochs,
)


In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

# List of image paths for different currency notes
image_paths = [
    r"R:\21VV1F0025\Data1\Test\Real Currency\50-2 (7).jpg",
    # Add more image paths for other currency notes here
]

for img_path in image_paths:
    # Load and preprocess the test image
    img = image.load_img(img_path, target_size=(img_height, img_width))
    test_image = image.img_to_array(img)
    test_image = np.expand_dims(test_image, axis=0)

    # Use your classifier model to make predictions
    result = Classifier.predict(test_image)
    predicted_class_index = np.argmax(result, axis=1)
    predicted_class = class_names[predicted_class_index[0]]  # Use without .item()

    # Extract the true class from the image path (assuming the path structure is consistent)
    true_class = img_path.split('\\')[-2]

    # Calculate and print accuracy
    is_correct = (true_class == predicted_class)
    accuracy = 1 if is_correct else 0  # 1 if correct, 0 if incorrect

    if accuracy == 1:
        print("REAL CURRENCY")
    else:
        print("FAKE CURRENCY")

    print("------------------------")

1/1 [==============================] - 0s 33ms/step
REAL CURRENCY
------------------------
